### GW tutorial 3: Generation of data and matched filtering

Author: Melissa Lopez

Email: m.lopez@uu.nl

In [1]:
%matplotlib inline
from pycbc.catalog import Merger
import pycbc.psd, pycbc.noise
import pylab
import matplotlib.pyplot as plt
import numpy as np
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector
import random

ModuleNotFoundError: No module named 'pycbc'

Previously, we have plotted the PSD of different detectors. Some PSDs are from previous runs (O3), while some others are the design sensitivity of future detectors.

The PSD characterizes the noise of the detector, so we can generate detector noise accordingly.

**Exercise 1:** Load the `aligo_O4high.txt` PSD to [generate](https://pycbc.org/pycbc/latest/html/pycbc.noise.html#pycbc.noise.gaussian.noise_from_psd) some Gaussian data.

_Hint:_ Minimum frequency is 10 Hz, sampling rate 4096 Hz and we want 20s of duration

In [ ]:
# Load the PSD for the O4 sensitivity curve (LIGO)
psd_o4 = pycbc.psd.read.from_txt("sensitivity_curves/aligo_O4high.txt")

# Set up parameters for the noise generation
duration = 20  # duration in seconds
sampling_rate = 4096  # sampling rate in Hz
min_freq = 10  # minimum frequency in Hz

# Generate Gaussian noise based on the PSD
noise = pycbc.noise.gaussian.noise_from_psd(duration=duration, psd=psd_o4, sample_rate=sampling_rate, min_freq=min_freq)


# Plot the generated noise
plt.figure(figsize=(12, 6))
plt.plot(noise.sample_times, noise)
plt.title('Simulated Gaussian Noise (O4 Sensitivity)')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.grid(True)
plt.show()


As you can see. the generated noise is "coloured" according to the detector noise. Now, we would like to add a simulated gravitational wave signal in our detector noise. 

**Exercise 2:** Using [this function](https://pycbc.org/pycbc/latest/html/pycbc.waveform.html#pycbc.waveform.waveform.get_td_waveform) to generate waveforms in time domain, provide the plus and cross polarization of a binary black hole merger of $m_{1} = m_{2} = 50$ at 2000 Mpc. Use the waveform approximant "IMRPhenomD". How does the waveform amplitude compare to the detector noise?

The waveform comes from the source, but it needs to be projected in the detector. [Project](https://pycbc.org/pycbc/latest/html/pycbc.detector.html#pycbc.detector.ground.Detector.project_wave) the waveform on LIGO Livingston (L1) at random sky-location and polarization. You can specify your detector with [this](https://pycbc.org/pycbc/latest/html/pycbc.detector.html#pycbc.detector.ground.Detector) function. How does the waveform change because of this projection?

_Hint:_ Right ascension range is $[0, 2\pi]$, declination is $[-2\pi, 2\pi]$ and polarization is $[0, 2\pi]$.

In [ ]:
# Parameters for the BBH waveform
m1 = m2 = 50  # masses in solar masses
distance = 2000  # distance in Mpc
approximant = "IMRPhenomD"  # waveform approximant

# Generate the waveform in the time domain
hp, hc = get_td_waveform(approximant=approximant,
                          mass1=m1, mass2=m2,
                          distance=distance,
                          f_lower=10,  # minimum frequency for the waveform
                          delta_t=1/sampling_rate)  # time step corresponding to the sample rate

# Plot the waveform
plt.figure(figsize=(12, 6))
plt.plot(hp.sample_times, hp, label="Plus polarization")
plt.plot(hc.sample_times, hc, label="Cross polarization")
plt.title('Binary Black Hole Merger Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.show()

# Initialize the LIGO Livingston detector
detector = Detector('L1')

# Random sky location (right ascension and declination) and polarization
ra = random.uniform(0, 2 * np.pi)  # Right Ascension
dec = random.uniform(-np.pi, np.pi)  # Declination
polarization = random.uniform(0, 2 * np.pi)  # Polarization angle

# Project the waveform onto the detector
hp_proj, hc_proj = detector.project_wave(hp, hc, ra, dec, polarization)

# Plot the projected waveform
plt.figure(figsize=(12, 6))
plt.plot(hp_proj.sample_times, hp_proj, label="Projected Plus polarization")
plt.plot(hc_proj.sample_times, hc_proj, label="Projected Cross polarization")
plt.title('Projected Binary Black Hole Merger Waveform on LIGO Livingston (L1)')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.show()

# Plot the noise along with the waveform
plt.figure(figsize=(12, 6))
plt.plot(noise.sample_times, noise, label="Simulated Detector Noise", color='gray', alpha=0.7)
plt.plot(hp_proj.sample_times, hp_proj, label="Projected Plus Polarization", color='blue')
plt.plot(hc_proj.sample_times, hc_proj, label="Projected Cross Polarization", color='red')
plt.title('Simulated Detector Noise with Projected BBH Waveform (L1)')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.grid(True)
plt.show()


**Exercise 2:** From before our detector noise is 20s long. Add the GW into noise after 5s. We also need to get the first 4s to estimate the PSD in the next excercise. Make an overlaid plot with the total data, the portion of data where the GW is added and the data needed to estimate the PSD. 

_Bonus:_ Can you also plot the GW signal? 


In [ ]:
# Parameters for the BBH waveform
m1 = m2 = 50  # masses in solar masses
distance = 2000  # distance in Mpc
approximant = "IMRPhenomD"  # waveform approximant

# Generate the waveform in the time domain
hp, hc = get_td_waveform(approximant=approximant,mass1=m1, mass2=m2,distance=distance,f_lower=10, delta_t=1/sampling_rate)
# Simulate Gaussian noise from the PSD 
psd_o4 = pycbc.psd.read.from_txt("sensitivity_curves/aligo_O4high.txt")
duration = 20  # total duration in seconds
sampling_rate = 4096  # sampling rate in Hz
min_freq = 10  # minimum frequency in Hz
noise = pycbc.noise.gaussian.noise_from_psd(duration=duration, psd=psd_o4, sample_rate=sampling_rate, min_freq=min_freq)

# Add the waveform to the noise after 5 seconds
gw_start_time = 5  # the GW signal starts at 5 seconds
gw_end_time = gw_start_time + len(hp) / sampling_rate  # end time based on the waveform duration

# Add the waveform to the noise after 5 seconds
total_data = noise.copy()
total_data[gw_start_time*sampling_rate : gw_end_time*sampling_rate] += hp

# Extract the first 4 seconds of data for PSD estimation
psd_data = noise[:4*sampling_rate]  # first 4 seconds of the noise

# Plotting everything

# Create the figure
plt.figure(figsize=(12, 6))

# Plot total data (noise + GW)
plt.plot(total_data.sample_times, total_data, label="Total Data (Noise + GW)", color='black')

# Plot the portion with the GW added (after 5s)
plt.plot(total_data.sample_times[gw_start_time*sampling_rate:], total_data[gw_start_time*sampling_rate:], label="GW Added After 5s", color='blue', alpha=0.7)

# Plot the data needed for PSD estimation (first 4s of noise)
plt.plot(psd_data.sample_times, psd_data, label="First 4s (PSD Estimation)", color='green', linestyle='--')

# Title and labels
plt.title("Simulated Detector Data: Noise + GW Signal")
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend()
plt.grid(True)
plt.show()

# Plot the GW signal separately (for visualization)
plt.figure(figsize=(12, 6))
plt.plot(hp.sample_times, hp, label="GW Signal (Plus Polarization)")
plt.plot(hc.sample_times, hc, label="GW Signal (Cross Polarization)")
plt.title("Gravitational Wave Signal (BBH Merger)")
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend()
plt.grid(True)
plt.show()


**Exercise 3:** In a proper search we do not really have the PSD handy. We want to estimate the PSD with Welch's method (see [here](https://ccrma.stanford.edu/~jos/sasp/Welch_s_Method.html) for details), but we want to use the 4s of the beginning where the GW is not present using [this function](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.welch). Plot the estimated PSD. Note that before we used a dummy whitening, and this one is a bit better.

What is the $\Delta_f$ of the estimated PSD? What is $\Delta_f$ of the data we want to whiten to see the GW signal?

_Hint:_ `seg_stride (int)` is usually half of `seg_len`.


In [ ]:
# Estimate the PSD using Welch's method for the first 4s of noise
seg_len = 4 * sampling_rate  # segment length in samples (4 seconds)
seg_stride = seg_len // 2  # usually half of seg_len

# Use Welch's method to estimate the PSD
estimated_psd = estimate.welch(psd_data, seg_len=seg_len, seg_stride=seg_stride)

# Plot the estimated PSD
plt.figure(figsize=(12, 6))
plt.loglog(estimated_psd.frequencies, estimated_psd, label='Estimated PSD (Welch)', color='blue')
plt.title('Estimated PSD using Welch\'s Method')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [strain^2/Hz]')
plt.grid(True)
plt.legend()
plt.show()

# Calculate the frequency resolution for the estimated PSD
delta_f_estimated = 1 / seg_len
print(f"Delta_f of the estimated PSD: {delta_f_estimated} Hz")

# Calculate the frequency resolution for the data used for whitening
delta_f_data = 1 / len(noise)
print(f"Delta_f of the data used for whitening: {delta_f_data} Hz")


Now that we have the PSD we need to interpolate it to match our data and then limit the filter length of 1 / PSD. After this, we can directly use this PSD to filter the data. Since the data has been highpassed above 12 Hz, and will have low values below this we need to informat the function to not include frequencies below this frequency. 

**Exercise 4:** Use the functions [interpolate](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.interpolate) and [inverse_spectrum_truncation](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.inverse_spectrum_truncation) to achieve a proper formatting of the PSD.

_Hint:_ `max_filter_len (int)` is $4 \times$ sampling_rate. Note that the original PSD has a minimum frequency of 12Hz.

In [ ]:
# Interpolate the PSD to match the data's frequency resolution
interpolated_psd = estimate.interpolate(estimated_psd, noise.sample_freq)

# Set the maximum filter length (4 * sampling_rate as the hint suggests)
max_filter_len = 4 * sampling_rate

# Use inverse spectrum truncation to limit the filter length
psd_filtered = estimate.inverse_spectrum_truncation(interpolated_psd, max_filter_len=max_filter_len)

# Filter the data using the adjusted PSD
whitened_data = total_data.whiten(psd=psd_filtered, low_frequency_cutoff=12)

# Plot the original data, whitened data, and the PSD after interpolation and truncation
plt.figure(figsize=(12, 6))

# Plot the original data (Noise + GW)
plt.plot(total_data.sample_times, total_data, label="Original Data (Noise + GW)", color='black')

# Plot the whitened data
plt.plot(whitened_data.sample_times, whitened_data, label="Whitened Data", color='red')

# Plot the interpolated PSD for reference
plt.loglog(interpolated_psd.frequencies, interpolated_psd, label="Interpolated PSD", color='blue')

plt.title("Filtered and Whitened Data vs Original Data")
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.legend()
plt.grid(True)
plt.show()


Now that the PSD is ready, we can whiten the data. Before we used a PyCBC function, but mathematically this is defined as

\begin{equation}
\tilde{d_w}(f) = \tilde{d}(f)/S_{n}^{-1/2}(f)
\end{equation}
where $\tilde{d}$ and $\tilde{d_w}(f)$ are the Fourier transform of the coloured data and whitened data, respectively.

**Exercise 5:** Whiten the data using the interpolated PSD. Crop 5s at the beginning and the end to avoid border effects (_aliasing_) and bandpass it as in the previous exercise. Can you see the GW signal?

In [ ]:
# Get the frequency domain representation of the original data
data_f = total_data.to_frequencyseries()

# Apply whitening: d_w(f) = d(f) / sqrt(Sn(f))
whitened_f = data_f / np.sqrt(interpolated_psd)

# Convert back to time domain
whitened_data = whitened_f.to_timeseries()

# Crop the data by 5s at the beginning and the end to avoid border effects
start_time = 5  # 5 seconds at the beginning
end_time = -5  # 5 seconds at the end
cropped_data = whitened_data.crop(start_time=start_time, end_time=end_time)

# Bandpass filter the data as done in previous exercises
cropped_data = cropped_data.lowpass_fir(250, order=512)
cropped_data = cropped_data.highpass_fir(30, order=512)

# Plot the whitened and bandpassed data
plt.figure(figsize=(12, 6))
plt.plot(cropped_data.sample_times, cropped_data, label="Whitened and Bandpassed Data")
plt.title("Whitened and Bandpassed Data with GW Signal")
plt.xlabel("Time [s]")
plt.ylabel("Strain")
plt.grid(True)
plt.legend()
plt.show()


So far, we buried a GW signal  $h(t)$ in stationary and Gaussian noise $n(t)$ with zero mean, such that $s(t) = h(t) + n(t)$. Given the optimal filter  $K(t)$, 

\begin{equation}
\label{eq:filteredsignal}
\hat{s} = \int_{-\infty}^{\infty}  K(t)s(t) dt = \int_{-\infty}^{\infty} \tilde{K}(f)^{*}\tilde{s}(f) df, \quad \text{ where }  \tilde{s}(f) = \int_{-\infty}^{\infty} s(t)e^{-2 \pi i ft}dt
\end{equation}
where $\hat{s}$ is the filtered value of $s(t)$, $^*$ represents the complex conjugate, and $\tilde{\cdot }\ $ the Fourier transform. 

The detection statistic that is maximised by the optimal filter $K(t)$  will be the SNR, defined as $\rho = S/N$. $S$ is the expected value of $\hat{s}$ when $h(t) \neq 0$, while $N$ is the squared root of the noise variance when $h(t) =0$. It can be demonstrated that the optimal filter $K(t)$ is the model of the GW signal itself, known as _template_.

The fundamental modelled detection technique is called "matched filtering", since the filter function is chosen to "match" the signal we are looking for. We can write the SNR between an unknown time series $s(t)$ and the template $h_{m}$ as

\begin{equation}
\label{eq:wienerscalar}
\rho =  4 \text{Re} \int_{0}^{\infty} \frac{\tilde{s}^{*}(f)}{S_{n}(f)} \tilde{h_{m}}(f) df. 
\end{equation}



**Exercise 6:** Use the [matched_filter](https://pycbc.org/pycbc/latest/html/pycbc.filter.html#pycbc.filter.matchedfilter.matched_filter) function to filter the coloured data with the template. Crop the SNR time series 5s at each side and plot it together with the whitened data. Where is the GW signal? Note that the y-axis of the whitened data is _amplitude_ and the y-axis of the SNR timeseries is _SNR_.

How much is the maximum of the absolute value of the SNR time series? If it is larger than 5 it will generate a "trigger" for further analysis. Will the trigger be generated?

Note that the parameter space of GW signals is vast, so to find these signals, we will need to create a template bank and do this process for thousands of templates. Then, this becomes a high-performance computing problem!

_Hint_: you need to [resize](https://pycbc.org/pycbc/latest/html/pycbc.types.html#pycbc.types.array.Array.resize) the template and use [cyclic_time_shift](https://pycbc.org/pycbc/latest/html/pycbc.types.html#pycbc.types.frequencyseries.FrequencySeries.cyclic_time_shift)

In [ ]:
m1 = 50  # mass of the first black hole (in solar masses)
m2 = 50  # mass of the second black hole (in solar masses)
distance = 2000  # distance in Mpc

# Generate the waveform using the "IMRPhenomD" approximant
hp, hc = get_td_waveform(approximant="IMRPhenomD", mass1=m1, mass2=m2, distance=distance)

# Define the template for matched filtering
template = hp

# Perform matched filtering
from pycbc.filter import matched_filter

# Perform the matched filter with the whitened data
snr_time_series = matched_filter(template, whitened_data)

# Crop the SNR time series by 5s at each side
cropped_snr = snr_time_series.crop(start_time=5, end_time=-5)

# Plot the whitened data and the SNR time series
plt.figure(figsize=(12, 6))

# Plot the whitened data
plt.subplot(2, 1, 1)
plt.plot(whitened_data.sample_times, whitened_data, label="Whitened Data", color='blue')
plt.title("Whitened Data with GW Signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.grid(True)
plt.legend()

# Plot the SNR time series
plt.subplot(2, 1, 2)
plt.plot(cropped_snr.sample_times, cropped_snr, label="SNR", color='red')
plt.title("SNR Time Series")
plt.xlabel("Time [s]")
plt.ylabel("SNR")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Find the maximum of the absolute value of the SNR time series
max_snr = np.max(np.abs(cropped_snr))
print(f"Maximum SNR: {max_snr}")

# Determine if a trigger is generated (SNR > 5)
trigger_generated = max_snr > 5
print(f"Trigger generated: {trigger_generated}")


Good job arriving at the end of the tutorial! This was a small peak at GW data analysis that I hope you found interesting. 

There is a bonus track exercise if you are bored, but we can also have a chat about some more GW data analysis if you prefer. 